## Cats vs Dogs Dataset

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [ ]:
path_cats_and_dogs = f"{getcwd()}/../tmp2/cats-and-dogs.zip"
shutil.rmtree('/tmp')

local_zip = path_cats_and_dogs
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


In [ ]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

1500
1500


In [ ]:
try:
    catsVdogs = os.path.join('/tmp', 'cats-v-dogs')
    os.mkdir(catsVdogs)
    
    train_dir = os.path.join('/tmp/cats-v-dogs', 'training')
    os.mkdir(train_dir)
    testing_dir = os.path.join('/tmp/cats-v-dogs', 'testing')
    os.mkdir(testing_dir)
    
    train_cats_dir = os.path.join(train_dir, 'cats')
    os.mkdir(train_cats_dir)
    train_dogs_dir = os.path.join(train_dir, 'dogs')
    os.mkdir(train_dogs_dir)
    
    testing_cats_dir = os.path.join(testing_dir, 'cats')
    os.mkdir(testing_cats_dir)
    testing_dogs_dir = os.path.join(testing_dir, 'dogs')
    os.mkdir(testing_dogs_dir)
except OSError:
    pass

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    source = os.listdir(SOURCE)
    source = random.sample(source,len(source))
    
    training_source = source[:int(SPLIT_SIZE * len(source))]
    for source_image in training_source:
        if os.path.getsize(SOURCE + source_image):
            copyfile(SOURCE + source_image, TRAINING +source_image)
            
    testing_source = source[int(SPLIT_SIZE * len(source)):]
    for source_image in testing_source:
        if os.path.getsize(SOURCE + source_image):
            copyfile(SOURCE + source_image, TESTING + source_image)


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

In [ ]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

1350
1350
150
150


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
TRAINING_DIR = train_dir
train_datagen = ImageDataGenerator(rescale = 1 / 255.0)

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                   batch_size = 10,
                                                   target_size = (150, 150),
                                                   class_mode = 'binary')

VALIDATION_DIR = testing_dir
validation_datagen = ImageDataGenerator(rescale = 1 / 255.0)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                   batch_size = 10,
                                                   target_size = (150, 150),
                                                   class_mode = 'binary')


Found 2700 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(train_generator,
                              epochs=2,
                              verbose=1,
                              validation_data=validation_generator)


Epoch 1/2
270/270 [==============================] - 38s 140ms/step - loss: 0.6863 - acc: 0.5833 - val_loss: 0.6410 - val_acc: 0.6033
Epoch 2/2
270/270 [==============================] - 37s 138ms/step - loss: 0.5878 - acc: 0.7019 - val_loss: 0.5813 - val_acc: 0.6900


In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')
